<a href="https://colab.research.google.com/github/knoppin1/DATA-690-WANG/blob/master/Assignement_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06 - Using Pandas
### Ken Noppinger (knoppin1@umbc.edu)
## Instructions
- This assignment is a re-do of assignment 04. Use Pandas and functions instead.
- Use the simple and convenient read_html() from Pandas to extract **HTML tables** from web pages. 
- Sample solution is here:
- https://github.com/wcj365/python-stats-dataviz/blob/master/pandas_apply.ipynb
- Make sure you user **Markdown cells** to document the process and its steps.
## Requirements
1. Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html
2. Find out how many HTML tables Pandas retrieves from the web page. Find out which dataframe contain the data and use it for further processing.
3. Display and explore the data (rows, columns, etc.) and determine the data quality (bad rows, bad columns, bad elements, null values, etc.)
4. Document the data quality issues using a Markdown cell so the reader understands what the problems are.
5. Write a function named cleanse_year() that takes a string as input and removes any asterisks in the string and return the cleansed string. Test the function by using test strings (for example, "1989*", "20*10")
6. Cleanse the column "Census Year" using the function cleanse_year() and Pandas's apply() function.
7. Repeat this process for column "Total Population", 	"Census Cost", 	"Average Cost Per Person". (define, test, and apply the function)
8. After all columns are cleansed, save the clenased dataframe to a file named "census_cost_cleansed.csv" using CSV format.
9. Use Pandas to read the saved cleansed file and explore to make sure it is clean. 
10. Updload both the Jupyter notebook and the cleanse data file to GitHub and submit the GitHub link in BlackBoard.


1. Write code to open the text file [census_cost.txt](census_cost.txt) and read all lines into a list named **"line_list"**. Print line_list.
2. Extract the first two lines and put them in a different list named "top2_list". You will need to use them later. Print the top2_list.
3. Put the rest of the lines (containing useful data elements) in a new list named "data_list". Print data_list.
4. Extract the column **"Census Year"** from data_list and assign them to a list named **year_list**. Remove the asterisk character from the last element "2010*". Print the cleansed year_list.
5. Extract the **"Total Population"** column from the data_list and assign them to a list named **"pop_list"**. Remove the "," from the numbers since Python doesn't recognize them. Print the cleansed "pop_list".
6. Extract the **"Census Cost"** column from the data_list and assign them to a list named **"cost_list"**. Remove the ",", and "\$", and "Billion".
Make sure to add the "0"s to the numbers from which you removed "Billion". Print the cleansed cost_list.  
7. Extract the **"Average Cost per Person"** column from the data_list and assign them to a list named **"avg_list"**. Remove the "cents", and "$".
Make sure to divide the numbers in cents by 100 so that all numbers are measured in dollar. Print the cleansed avg_list.
8. Coalesce the cleansed data and save them to a text file named **"census_cost.csv"**. The new file should look similar to the original source file except that it is in comma-delimited format and the numbers have been cleansed. The top two lines from the original file should be retained in the new file.

---

In [1]:
import pandas as pd

## **1 -  Read HTML tables**
- Use Pandas read_html() function to retrive the HTML tables as dataframes from the web page: https://www.genealogybranches.com/censuscosts.html
-  Read all HTML tables into a list named **census_cost_data**. 



In [2]:
census_cost_data = pd.read_html("https://www.genealogybranches.com/censuscosts.html")
type(census_cost_data)

list

## **2 - Determine # of HTML tables Pandas retrieves from the web page.**

In [3]:
len(census_cost_data)

2

In [4]:
census_cost_data

[              0                 1               2                        3
 0   Census Year  Total Population     Census Cost  Average Cost Per Person
 1          1790           3929214         $44,377               1.13 cents
 2          1800           5308483         $66,109               1.24 cents
 3          1810           7239881        $178,445               2.46 cents
 4          1820           9633822        $208,526               2.16 cents
 5          1830          12866020        $378,545               2.94 cents
 6          1840          17069458        $833,371               4.88 cents
 7          1850          23191876      $1,423,351               6.14 cents
 8          1860          31443321      $1,969,377               6.26 cents
 9          1870          38558371      $3,421,198               8.87 cents
 10         1880          50155783      $5,790,678              11.54 cents
 11         1890          62979766     $11,547,127              18.33 cents
 12         

The census data is a list containing two dataframes.  The census data of interest is in the first datframe, so save that as **census_df**

In [5]:
census_df = census_cost_data[0]
census_df.head(5)

,0,1,2,3
0,Census Year,Total Population,Census Cost,Average Cost Per Person
1,1790,3929214,"$44,377",1.13 cents
2,1800,5308483,"$66,109",1.24 cents
3,1810,7239881,"$178,445",2.46 cents
4,1820,9633822,"$208,526",2.16 cents


First row contains column headers, so make that row the header for the dataframe

In [6]:
census_df.columns = census_df.iloc[0]
census_df.drop([0], inplace=True)
census_df.reset_index(drop=True, inplace=True)
census_df.head(5)

,Census Year,Total Population,Census Cost,Average Cost Per Person
0,1790,3929214,"$44,377",1.13 cents
1,1800,5308483,"$66,109",1.24 cents
2,1810,7239881,"$178,445",2.46 cents
3,1820,9633822,"$208,526",2.16 cents
4,1830,12866020,"$378,545",2.94 cents


## **3 - Explore the data** 

### Number of rows and columns

In [7]:
census_df.shape

(23, 4)

23 rows and 4 columns

### Check dataframe for nulls

Reference - https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

In [8]:
census_df.isnull().values.any()

False

Dataframe doesn't contain any nulls requiring cleanup.

### Check dataframe columns and rows for bad data

In [9]:
census_df.dtypes

0
Census Year                object
Total Population           object
Census Cost                object
Average Cost Per Person    object
dtype: object

Note the columns are all objects.  They will need to be cleaned up and cast to proper data dtypes.

In [10]:
census_df.describe()

,Census Year,Total Population,Census Cost,Average Cost Per Person
count,23,23,23,23
unique,23,23,23,23
top,1900,7239881,"$3,421,198",60.44 cents
freq,1,1,1,1


### Bad rows